In [1]:
BERT_MODEL = 'bert-base-multilingual-cased'

# CustomBert with adapters no discodisco

In [2]:
from transformers.models.bert.modeling_bert import BertModel
import torch
# import copy
# from typing import Any, Dict, Optional
# from transformers import XLMRobertaModel, XLMRobertaConfig
import torch.nn as nn
# from tensorflow.keras.layers import TimeDistributed
# from featurefulbertembedder_custom_xlmr import FeaturefulBertEmbedder
# from featureful_bert_custom_xlmr import get_combined_feature_tensor_2 as get_combined_feature_tensor_forward
from transformers import AutoConfig, AutoAdapterModel


class CustomBERTModel(nn.Module):
    def __init__(self, num_labels):
          super(CustomBERTModel, self).__init__()
          self.num_classes = num_labels
        #   self.feature_list = mnli_dataset.feature_list
          print('ASSIGN:', self.num_classes)

          self.embedder = self.create_featureful_bert()
        #   self.encoder = CustomPooler2()
        #   self.module1 = MyModule(self.feature_list)
        #   self.dropout1 = nn.Dropout(p=0.0)
        #   self.dropout_decoder = nn.Dropout(p=0.5)
          self.relation_decoder = nn.Linear(792, self.num_classes)

    def forward(self, pair_token_ids, attention_mask, feat):
        direction_tensor = feat['dir']
        embedded_sentence = self.embedder(token_ids=pair_token_ids, #featurefulmebedder
                        mask=attention_mask, 
                        # type_ids=token_type_ids,
                        segment_concat_mask = None,
                        direction_tensor = direction_tensor,
                        feature_list = self.feature_list,
                        features = feat)
        # mask = token_type_ids
        bertpooler_output = self.encoder(tokens=embedded_sentence, mask=None)
        feat = self.convert_to_feature_list(feat)
        feat = self.dropout1(feat)
        feat = self.module1(feat)
        try:
            feat_concat = torch.concat((bertpooler_output, feat),-1)
        except:
            print(bertpooler_output.shape, feat.shape)
            raise ValueError()
        if feat_concat.shape[-1]!=792: print(feat_concat.shape)
        assert feat_concat.shape[-1] == 792
        feat_concat = self.dropout1(feat_concat)
        # feat_concat = self.dropout_decoder(feat_concat)
        linear1_output = self.relation_decoder(feat_concat)
        return linear1_output


    # def create_featureful_bert(self):
    #     featureful_bert = FeaturefulBertEmbedder(model_name = BERT_MODEL,
    #                             hidden_activation_allen = 'gelu',
    #                             feature_list = self.feature_list, 
    #                             vocab=mnli_dataset.vocab)
    #     return featureful_bert

    def create_featureful_bert(self):
        self.configadapter = AutoConfig.from_pretrained(
            BERT_MODEL,
        )
        model = AutoAdapterModel.from_pretrained(
            BERT_MODEL,
            config=self.configadapter,
        )
        return model

    def convert_to_feature_list(self, feat):
        feature_linear = [feat[feature_name] for feature_name in self.feature_list]
        feature_linear = torch.stack(feature_linear, dim=-1)
        return feature_linear
        

model = CustomBERTModel(23)
print(model)


/home/VD/kaveri/anaconda3/envs/discodapter/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ASSIGN: 23


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertAdapterModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CustomBERTModel(
  (embedder): BertAdapterModel(
    (shared_parameters): ModuleDict()
    (bert): BertModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict()
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(
                  in_features=768, out_features=768, bias=True
                  (loras): ModuleDict()
                )
                (key): Linear(
                  in_features=768, out_features=768, bias=True
                  (loras): ModuleDict()
                )
 

In [3]:
from transformers import AdapterConfig
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)

model.embedder.load_adapter("en/wiki@ukp", config=lang_adapter_config)
model.embedder.load_adapter("de/wiki@ukp", config=lang_adapter_config)

print(model)

CustomBERTModel(
  (embedder): BertAdapterModel(
    (shared_parameters): ModuleDict()
    (bert): BertModel(
      (shared_parameters): ModuleDict()
      (invertible_adapters): ModuleDict(
        (en): NICECouplingBlock(
          (F): Sequential(
            (0): Linear(in_features=384, out_features=192, bias=True)
            (1): Activation_Function_Class(
              (f): ReLU()
            )
            (2): Linear(in_features=192, out_features=384, bias=True)
          )
          (G): Sequential(
            (0): Linear(in_features=384, out_features=192, bias=True)
            (1): Activation_Function_Class(
              (f): ReLU()
            )
            (2): Linear(in_features=192, out_features=384, bias=True)
          )
        )
        (de): NICECouplingBlock(
          (F): Sequential(
            (0): Linear(in_features=384, out_features=192, bias=True)
            (1): Activation_Function_Class(
              (f): NewGELUActivation()
            )
            (

# Custom BERT discodisco with adapter inside

In [4]:
from transformers.models.bert.modeling_bert import BertModel
import torch
# import copy
# from typing import Any, Dict, Optional
# from transformers import XLMRobertaModel, XLMRobertaConfig
import torch.nn as nn
# from tensorflow.keras.layers import TimeDistributed
from featurefulbertembedder_custom2 import FeaturefulBertEmbedder
from featureful_bert_custom2 import get_combined_feature_tensor_2 as get_combined_feature_tensor_forward
from transformers import AutoConfig, AutoAdapterModel


class CustomBERTModel(nn.Module):
    def __init__(self, num_labels):
          super(CustomBERTModel, self).__init__()
          self.num_classes = num_labels
        #   self.feature_list = mnli_dataset.feature_list
          print('ASSIGN:', self.num_classes)

          self.embedder = self.create_featureful_bert()
        #   self.encoder = CustomPooler2()
        #   self.module1 = MyModule(self.feature_list)
        #   self.dropout1 = nn.Dropout(p=0.0)
        #   self.dropout_decoder = nn.Dropout(p=0.5)
          self.relation_decoder = nn.Linear(792, self.num_classes)

    def forward(self, pair_token_ids, attention_mask, feat):
        direction_tensor = feat['dir']
        embedded_sentence = self.embedder(token_ids=pair_token_ids, #featurefulmebedder
                        mask=attention_mask, 
                        # type_ids=token_type_ids,
                        segment_concat_mask = None,
                        direction_tensor = direction_tensor,
                        feature_list = None,
                        features = feat)
        # mask = token_type_ids
        bertpooler_output = self.encoder(tokens=embedded_sentence, mask=None)
        feat = self.convert_to_feature_list(feat)
        feat = self.dropout1(feat)
        feat = self.module1(feat)
        try:
            feat_concat = torch.concat((bertpooler_output, feat),-1)
        except:
            print(bertpooler_output.shape, feat.shape)
            raise ValueError()
        if feat_concat.shape[-1]!=792: print(feat_concat.shape)
        assert feat_concat.shape[-1] == 792
        feat_concat = self.dropout1(feat_concat)
        # feat_concat = self.dropout_decoder(feat_concat)
        linear1_output = self.relation_decoder(feat_concat)
        return linear1_output


    def create_featureful_bert(self):
        featureful_bert = FeaturefulBertEmbedder(model_name = BERT_MODEL,
                                hidden_activation_allen = 'gelu',
                                feature_list = None, 
                                vocab=mnli_dataset.vocab)
        return featureful_bert

    # def create_featureful_bert(self):
    #     self.configadapter = AutoConfig.from_pretrained(
    #         BERT_MODEL,
    #     )
    #     model = AutoAdapterModel.from_pretrained(
    #         BERT_MODEL,
    #         config=self.configadapter,
    #     )
    #     return model

    def convert_to_feature_list(self, feat):
        feature_linear = [feat[feature_name] for feature_name in self.feature_list]
        feature_linear = torch.stack(feature_linear, dim=-1)
        return feature_linear
        

model = CustomBERTModel(23)
print(model)


2023-01-18 16:31:22.143445: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 16:31:22.298731: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-10.2/targets/x86_64-linux/lib/:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-18 16:31:22.298748: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up o

ASSIGN: 23


NameError: name 'mnli_dataset' is not defined

In [ ]:
raise ValueError()

In [ ]:
model = BertModel.from_pretrained(BERT_MODEL)
print(model)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (shared_parameters): ModuleDict()
  (invertible_adapters): ModuleDict()
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (key): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (value): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (dropout): Dropout(p=0.1, inplace=Fals

In [ ]:
# self.encoder = BertEncoder(config)
# self.pooler = BertPooler(config) if add_pooling_layer else None

In [ ]:
from transformers.models.bert.modeling_bert import BertEmbeddings, BertConfig
from featurefulbertembedder_custom2 import FeaturefulBert

transformer_model = FeaturefulBert.from_pretrained(pretrained_model_name_or_path=BERT_MODEL)
print(transformer_model)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing FeaturefulBert: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing FeaturefulBert from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FeaturefulBert from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FeaturefulBert(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (key): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (value): Linear(
              in_features=768, out_features=768, bias=True
              (loras): ModuleDict()
            )
            (dropout): Dropout(p=0.1, inplace=False)
            (prefix_tuning): PrefixTuningShim(
              (pref